<p style="font-family: Arial; font-size:3.75em;color:purple; font-style:bold"><br>
Sudoku
</p><br>
<strong>The ultimate spoiler for a time-wasting game ;)</strong>
<p>Possible improvements:</p>
- JS front-end
- randomly create a new game
- identify the level of difficulty of a game

<img src="https://automatetheboringstuff.com/images/000037.jpg">

### Import or create grid

In [1]:
import pandas as pd

In [2]:
#By default, the grid is converted to a table.
# dict_grid={
#     'A':[5,6,'',8,4,7,'','',''],
#     'B':[3,'',9,'','','',6,'',''],
#     'C':['','',8,'','','','','',''],
#     'D':['',1,'','',8,'','',4,''],
#     'E':[7,9,'',6,'',2,'',1,8],
#     'F':['',5,'','',3,'','',9,''],
#     'G':['','','','','','',2,'',''],
#     'H':['','',6,'','','',8,'',7],
#     'I':['','','',3,1,6,'',5,9]
    
# }
# grid=pd.DataFrame(data=dict_grid)
# grid

In [3]:
#Alternative: Open a template for user input. Save the file when done inputting a sudoku grid.
#This takes a while as it will open Excel
# import csv
# import subprocess

# with open('sudoku.csv', 'w',newline='') as csvfile:
#     filewriter = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
#     filewriter.writerow(['',0,1,2,3,4,5,6,7,8])
#     for i in range(0,9):
#         filewriter.writerow([i])
# subprocess.call("explorer sudoku.csv")

In [4]:
grid=pd.read_csv('sudoku2.csv',index_col=0)
grid=grid.apply(pd.to_numeric).fillna('')
grid

,0,1,2,3,4,5,6,7,8
0,,4,,2,,1,,6,
1,,,,,,,,,
2,9,,5,,,,3,,7
3,,,,,,,,,
4,5,,7,,8,,1,,4
5,,1,,,,,,9,
6,,,1,,,,6,,
7,,,,7,,5,,,
8,6,,8,9,,4,5,,3


### Define variables for the 3x3 blocks
<p>Insert into a function: to be recalculated every time a new value is entered in the grid</p>

In [5]:
#example:
upper_left=grid.iloc[:3,:3]
#the subgrids will be numbered from 0 to 8, from top-left to bottom-right. upper_left corresponds to sub_grids[0]
#remove the empty cells, we need blocks only to generate a set of digits.
def content_blocks(grid):
    blocks=[]
    i=1
    while i <10:
        col=0
        while col<9:
            if i in [1,2,3]:
                row=0
            elif i in [4,5,6]:
                row=3
            elif i in [7,8,9]:
                row=6
            sub=grid.iloc[row:row+3,col:col+3]
            sub = set(filter(None, sub.values.flatten()))
            blocks.append(sub)
            col+=3
            i+=1
    return blocks
content_blocks(grid)[6]

{1.0, 6.0, 8.0}

### Define a class Cell with a value and block number

In [19]:
class Cell():
    def __init__(self, grid, x, y):
        '''grid is a pd.dataFrame
        x and y are integers within 0 and 8'''
        self.x=x
        self.y=y
        self.value = grid.iloc[x][y]
        self.id=str(x)+str(y)
    def find_block(self):
        if self.x<3:
            self.list_grids=[0,1,2]
        elif self.x<6:
            self.list_grids=[3,4,5]
        else:
            self.list_grids=[6,7,8]
        if self.y<3:
            return self.list_grids[0]
        elif self.y<6:
            return self.list_grids[1]
        else:
            return self.list_grids[2]
    def uncertain_cells_in_section(self,dictionary):
        possible_keys=sorted(dictionary.keys())
        uncertain_in_row=[x for x in possible_keys if x[0]==str(self.x) and x!=self.id]
        uncertain_in_column=[y for y in possible_keys if y[1]==str(self.y) and y!=self.id]
        uncertain_in_block=[z for z in possible_keys if z in cells_in_block(self.find_block()) and z!=self.id]
        return uncertain_in_row,uncertain_in_column,uncertain_in_block
ex_cell=Cell(grid,3,3)
print(ex_cell.value)
print(ex_cell.find_block())


4


### Algorithms
[Reference](https://www.kristanix.com/sudokuepic/sudoku-solving-techniques.php)
- 1 <strong>Sole candidate</strong>: cells that can only be filled by one digit (in a row + column + block)
- 2 <strong>Unique candidate</strong>: the cell has several options but is the only one in the row/column/block to have one digit as an option
- 3 <strong>Block and column/row interaction</strong>: if a digit can only be on 1 row or column on a block then we can eliminate it from the possibilities of the other blocks for that row or column.
- 4 <strong>Block/block interaction</strong>: if on 2 aligned blocks a digit can be in only 2 rows then we can eliminate it from these two rows on the third block
- 5 <strong>Naked subset</strong>: when 2 cells in a row/column/block are the only ones to admit the same only two digits, even if these digits are a possibility on other cells (so they can be removed from the options for these other cells, and the list of options for the 2 cells are reduced to these digits only)
- 6 <strong>X-Wing</strong>: look at cells disposed at corners of a rectangle of any size (each corner in a different block): if one digit is an option in each corner cell (and also on their respective rows (or columns)), and if in addition the digit can only be in one of the corners on each side of the rectangle: then the digit can only be in the corner and can be removed from the options for other cells in the rectangle. More generally, <strong>swordfish</strong>: when candidate cells for a specific digit happen to allow for this option on separate rows or columns, but specifically 2 times on each row & 2 times on each column; and at the same time the digit is an option only in all of the rows (or all of the columns). Then it can be removed from all the columns (or rows). To recognize this pattern, see if it is possible to loop (using only vertical or horizontal lines) between all the cells containing a candidate number.
- 7 <strong>Forcing chain</strong>: guess an option for a cell and write down how it would affect other cells. Then guess another option and see if other cells would resolve in the same digit anyway.

In [9]:
def sole_candidate(grid,cell,dict_possibles,filled_with_possibles):
    forbidden=[]
    if cell.id in dict_possibles:
        possible=dict_possibles[cell.id]
    else:
        possible=list(range(1,10))
    #list digits in the row
    forbidden=set(filter(None,grid.loc[cell.x]))
    #list digits in the column
    forbidden.update(filter(None,set(list(grid[grid.columns[cell.y]]))))
    #list digits in the subgrid
    forbidden.update(content_blocks(grid)[cell.find_block()])
    #remove forbidden digits from a set of all possible digits (from 1 to 9)
    possible=set(possible).difference(forbidden)
    if len(possible)==1:
        grid.iloc[cell.x][cell.y]=list(possible)[0]
    else:
        dict_possibles[cell.id]=possible
        filled_with_possibles.iloc[cell.x][cell.y]=list(possible)
        if possible==set():
            print('Error, this grid cannot be solved: check the grid pre-filled with options, one cell has no options left')
            return 'alert'   
#sole_candidate(grid,Cell(grid,2,5))

In [10]:
def solve(grid):
    dict_possibles={}
    filled_ori=len(list(filter(None,grid.values.flatten())))
    print(filled_ori)
    filled=0
    while filled<81:
        filled_with_possibles=grid.copy()
        for row in range(0,9):
            for col in range(0,9):
                alert=False
                cell=Cell(grid,row,col)
                if cell.value=='':
                    result=sole_candidate(grid,cell,dict_possibles,filled_with_possibles)                    
                    if result=='alert':
                        return dict_possibles, filled_with_possibles
        if len(list(filter(None,grid.values.flatten())))==filled:
            print("Sorry, cannot solve this puzzle, try another method")
            return dict_possibles, filled_with_possibles
        filled=len(list(filter(None,grid.values.flatten())))
        print(filled)
    return grid,0

In [11]:
test=grid.copy()
#dict_possibles, filled_with_possibles=solve(test)

In [12]:
#filled_with_possibles

<img src="sole_candidate.png">

In [13]:
def cells_in_block(block_number):
    if block_number in [0,3,6]:
        col=0
    elif block_number in [1,4,7]:
        col=3
    else:
        col=6
    if block_number in [0,1,2]:
        row=0
    elif block_number in [3,4,5]:
        row=3
    else:
        row=6
    return [str(r)+str(c) for r in range(row,row+3) for c in range(col,col+3)]

In [14]:
def unique_candidate(grid,cell,dict):
    possible=dict[cell.id]
    uncertain_row,uncertain_col,uncertain_block=cell.uncertain_cells_in_section(dict)
    for section in uncertain_row,uncertain_col,uncertain_block:
        options=[list(dict[key]) for key in section]
        options=set([item for sublist in section for item in sublist])
        possible_section=set(possible).difference(options)
        if bool(possible_section):
            grid.iloc[cell.x][cell.y]=next(iter(possible_section))

In [15]:
def solve1(grid):
    dict_possibles={}
    filled_ori=len(list(filter(None,grid.values.flatten())))
    print(filled_ori)
    filled=0
    while filled<81:
        filled_with_possibles=grid.copy()
        for row in range(0,9):
            for col in range(0,9):
                alert=False
                cell=Cell(grid,row,col)
                if cell.value=='':
                    result=sole_candidate(grid,cell,dict_possibles,filled_with_possibles)
                    if result=='alert':
                        return dict_possibles, filled_with_possibles
        if filled==len(list(filter(None,grid.values.flatten()))):
            for row in range(0,9):
                for col in range(0,9):
                    cell=Cell(grid,row,col)
                    if cell.value=='':
                        unique_candidate(grid,cell,dict_possibles)
            if filled==len(list(filter(None,grid.values.flatten()))):
                print("Sorry, cannot solve this puzzle, try another method")
                break
        filled=len(list(filter(None,grid.values.flatten())))
        print(filled)
    return grid,0

In [20]:
test3=grid.copy()
dict_possibles, filled_with_possibles=solve1(test3)
test3

25
26
81


,0,1,2,3,4,5,6,7,8
0,8,4,3,2,9,1,8,6,8
1,8,8,2,3,3,3,8,1,1
2,9,8,5,8,4,8,3,8,7
3,8,2,9,1,1,2,8,2,8
4,5,9,7,3,8,9,1,2,4
5,8,1,2,3,2,2,8,9,8
6,2,2,1,8,2,8,6,8,8
7,2,9,9,7,1,5,8,8,8
8,6,2,8,9,1,4,5,1,3
